# LU decomposition - pivoting

The previous notebook presents an algorithm for computing the [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition). The algotithm, however, does not apply any pivoting scheme for preventing spurious errors due to numerical instabilities. This notebook shows how the partial pivoting changes our previous algorithm for computing the LU decomposition.

Let's recall the solution of a linear system by using the Gaussian elimination with partial pivoting:

$$
\begin{array}{ccccc}
\mathbf{A}^{(0)} = \mathbf{A} & & & \mathbf{y}^{(0)} = \mathbf{y} \\\\
\mathbf{A}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{P}^{(1)}\mathbf{A}^{(0)} & & &
\mathbf{y}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{P}^{(1)}\mathbf{y}^{(0)} \\\\
\mathbf{A}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{P}^{(2)}\mathbf{A}^{(1)} & & &
\mathbf{y}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{P}^{(2)}\mathbf{y}^{(1)}
\end{array} \: ,
$$

where $\mathbf{P}^{(k)}$ is the permutation matrix used to interchange the rows and perform the partial pivoting.

At the end of this algorithm, the original matrix $\mathbf{A}$ is transformed into an upper triangular matrix $\mathbf{A}^{(2)}$.

Notice that, according to the algorithm, the matrix $\mathbf{A}^{(2)}$ can be written as follows:

$$
\begin{split}
\mathbf{A}^{(2)} 
&= 
\left( \mathbf{I} - \mathbf{M}^{(2)} \right) \mathbf{P}^{(2)} \,
\left( \mathbf{I} - \mathbf{M}^{(1)} \right) \mathbf{P}^{(1)} \,
\mathbf{A} \\
&=
\underbrace{\left( \mathbf{I} - \mathbf{M}^{(2)} \right)}_{\mathbf{Q}^{(2)}} \,
\underbrace{\mathbf{P}^{(2)} \left( \mathbf{I} - \mathbf{M}^{(1)} \right) \mathbf{P}^{(-2)}}_{\mathbf{Q}^{(1)}} \,
\mathbf{P}^{(2)} \mathbf{P}^{(1)} \,
\mathbf{A} \\
&= \mathbf{Q} \mathbf{P} \mathbf{A}
\end{split}
$$

wHere $\mathbf{P}^{(-k)} \equiv \left( \mathbf{P}^{(k)} \right)^{-1}$, $\mathbf{P} = \mathbf{P}^{(2)} \, \mathbf{P}^{(1)}$ and $\mathbf{Q} = \mathbf{Q}^{(2)} \, \mathbf{Q}^{(1)}$.

The permuted matrix $\mathbf{P}\mathbf{A}$ can be written as follows:

$$
\mathbf{P}\mathbf{A} = \mathbf{Q}^{-1} \, \mathbf{A}^{(2)} \: ,
$$

where $\mathbf{Q}^{-1} = \mathbf{Q}^{(-1)} \, \mathbf{Q}^{(-2)}$, $\mathbf{Q}^{(-k)} \equiv \left( \mathbf{Q}^{(k)} \right)^{-1}$. As in the previous class, we need to know how to define a generic $\mathbf{Q}^{(-k)}$. Let's first define a generic $\mathbf{Q}^{(k)}$ as follows:

$$
\mathbf{Q}^{(k)} = \mathbf{P}^{(N-1)} \cdots \mathbf{P}^{(k+2)}\mathbf{P}^{(k+1)} \,
\left( \mathbf{I} - \mathbf{M}^{(k)} \right) \,
\mathbf{P}^{-(k+1)}\mathbf{P}^{-(k+2)} \cdots \mathbf{P}^{-(N-1)} \: .
$$

Then, a generic $\mathbf{Q}^{(-k)}$ is given by:

$$
\mathbf{Q}^{(-k)} = \mathbf{P}^{-(N-1)} \cdots \mathbf{P}^{-(k+2)}\mathbf{P}^{-(k+1)} \,
\left( \mathbf{I} + \mathbf{M}^{(k)} \right) \,
\mathbf{P}^{(k+1)}\mathbf{P}^{(k+2)} \cdots \mathbf{P}^{(N-1)} \: .
$$

In [2]:
PAREI AQUI

SyntaxError: invalid syntax (<ipython-input-2-29da87566340>, line 1)

We can verify this equation by showing that the product $\mathbf{Q}^{(k)} \left( \mathbf{Q}^{(k)}\right)^{-1}$ equals to the identity, as follows:

$$
\begin{split}
\mathbf{Q}^{(k)} \left( \mathbf{Q}^{(k)}\right)^{-1}
&= \left[ \mathbf{I} - \mathbf{t}^{(k)} \otimes \left( \mathbf{u}^{(k)} \right)^{\top} \right] \,
\left[ \mathbf{I} + \mathbf{t}^{(k)} \otimes \left( \mathbf{u}^{(k)} \right)^{\top} \right] \\
&= \mathbf{I} - \mathbf{t}^{(k)} \otimes \left( \mathbf{u}^{(k)} \right)^{\top}
+ \mathbf{t}^{(k)} \otimes \left( \mathbf{u}^{(k)} \right)^{\top} -
\mathbf{t}^{(k)} \otimes \left( \mathbf{u}^{(k)} \right)^{\top} \, \mathbf{t}^{(k)} \otimes \left( \mathbf{u}^{(k)} \right)^{\top} \\
&= \mathbf{I} - 
\mathbf{t}^{(k)} \otimes 
\underbrace{ \left( \mathbf{u}^{(k)} \right)^{\top} \mathbf{t}^{(k)} }_{= \, 0}
\otimes \left( \mathbf{u}^{(k)} \right)^{\top} \\
&= \mathbf{I}
\end{split} \: .
$$

The third row of the equation above uses the fact that $\left( \mathbf{u}^{(k)} \right)^{\top} \mathbf{t}^{(k)} = 0$. This property can be easily verified by noting that, while the $k$th element of $\mathbf{u}^{(k)}$ is its only nonzero element, the $k$th element of $\mathbf{t}^{(k)}$ is equal to zero. Consequently, the dot product equals to zero.

By using this result, we can write the matrix $\mathbf{Q}^{-1}$ as follows:

$$
\begin{split}
\mathbf{Q}^{-1}
&= \left[ \mathbf{I} + \mathbf{t}^{(1)} \otimes \left( \mathbf{u}^{(1)} \right)^{\top} \right]
   \left[ \mathbf{I} + \mathbf{t}^{(2)} \otimes \left( \mathbf{u}^{(2)} \right)^{\top} \right] \\
&= \mathbf{I} + \mathbf{t}^{(1)} \otimes \left( \mathbf{u}^{(1)} \right)^{\top} +
   \mathbf{t}^{(2)} \otimes \left( \mathbf{u}^{(2)} \right)^{\top} + 
   \mathbf{t}^{(1)} \otimes 
   \underbrace{ \left( \mathbf{u}^{(1)} \right)^{\top} \mathbf{t}^{(2)}}_{= \, 0}
   \otimes \left( \mathbf{u}^{(2)} \right)^{\top} \\
&= \mathbf{I} + \mathbf{t}^{(1)} \otimes \left( \mathbf{u}^{(1)} \right)^{\top} +
   \mathbf{t}^{(2)} \otimes \left( \mathbf{u}^{(2)} \right)^{\top} \\
&= 
\left[ \begin{array}{ccc}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array} \right] +
\left[ \begin{array}{ccc}
0 & 0 & 0 \\
t_{2}^{(1)} & 0 & 0 \\
t_{3}^{(1)} & 0 & 0
\end{array} \right] +
\left[ \begin{array}{ccc}
0 & 0 & 0 \\
0 & 0 & 0 \\
0 & t_{3}^{(2)} & 0
\end{array} \right] \\
&=
\left[ \begin{array}{ccc}
1 & 0 & 0 \\
t_{2}^{(1)} & 1 & 0 \\
t_{3}^{(1)} & t_{3}^{(2)} & 1
\end{array} \right]
\end{split} \: ,
$$

which is a unit **L**ower triangular matrix containing the Gauss multipliers. Because of that, $\mathbf{Q}^{-1}$ is commonly defined as $\mathbf{L}$. Similarly, the **U**pper triangular matrix $\mathbf{A}^{(2)}$ is defined as $\mathbf{U}$ and the original matrix $\mathbf{A}$ is factored as follows:

$$
\mathbf{A} = \mathbf{L} \, \mathbf{U} \: .
$$

The mathematical development presented here is valid for $3 \times 3$ matrices, however, it can be easily generalized to $N \times N$ matrices as well.

The LU decomposition described above can be implemented as follows:

    N = y.size
    C = np.copy(A)
    
    for i = 1:N-1
        
        # assert the pivot is nonzero
        assert C[i,i] != 0., 'null pivot!'
        
        # calculate the Gauss multipliers and store them 
        # in the lower part of C
        C[i+1:,i] = C[i+1:,i]/C[i,i]
        
        # zeroing of the elements in the ith column
        C[i+1:,i+1:] = C[i+1:,i+1:] - outer(C[i+1:,i], C[i,i+1:])
        
    return C

This algorithm receives a square matrix $\mathbf{A}$ and returns a matrix $\mathbf{C}$ containing the triangular matrices $\mathbf{L}$ and $\mathbf{U}$. The elements of $\mathbf{L}$, except the unitary elements of its main diagonal, are stored below the main diagonal of $\mathbf{C}$. The elements of $\mathbf{U}$ are stored in the upper part of $\mathbf{C}$, including its main diagonal.

## Solving a linear system by using the LU decomposition

Once the triangular matrices $\mathbf{L}$ and $\mathbf{U}$ are calculated, we may use them to solve a linear system $\mathbf{A} \mathbf{x} = \mathbf{y}$. Let's first substitute the LU decomposition into the linear system:

$$
\begin{split}
\mathbf{A} \mathbf{x} &= \mathbf{y} \\
\mathbf{L} \mathbf{U} \mathbf{x} &= \mathbf{y}
\end{split} \: .
$$

This equation shows that the original linear system can be represented by two triangular systems:

$$
\begin{split}
\mathbf{L}\mathbf{w} &= \mathbf{y} \\
\mathbf{U}\mathbf{x} &= \mathbf{w}
\end{split} \: ,
$$

where $\mathbf{w}$ is a dummy variable. Therefore, the linear system can be solved in two steps: 1) solve the lower triangular system for $\mathbf{w}$ and then 2) use it to solve the upper triangular system for $\mathbf{x}$.